# PRÁTICA GUIADA - Data Wrangling.

### PARTE I: Limpeza e transformação de dados.

#### Esta prática tem como objetivo fornecer um catálogo de métodos e funções em Pandas e Python que podem ser úteis ao lidar com tarefas de limpeza de dados. 

#### Em geral, podemos identificar seis tipos de tarefas ou operações que são aplicadas aos dados no estágio de limpeza.

1. Padronização de categorias (homogeneização).
2. Resolução de problemas de formato.
3. Atribuição de formato adequados (dtype).
4. Correção de valores incorretos.
5. Preencher dados faltantes (missing data imputation).
6. Organização correta do conjunto de dados (tidy data).

#### As funções e métodos apresentados abrangem uma ou várias dessas operações.

### Remover duplicados

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4, 'k2': [1, 1, 2, 3, 3, 3, 4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,3
6,two,4


#### O método  [`.duplicated()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) retorna um booleano identificando os casos duplicados.

In [3]:
print('Registros duplicados:', data.duplicated().sum())
data.duplicated()

Registros duplicados: 3


0    False
1     True
2    False
3    False
4     True
5     True
6    False
dtype: bool

#### Já o método [`drop_duplicates()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) retorna o `DataFrame` sem os casos duplicados.

In [4]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
6,two,4


In [5]:
data[~data.duplicated()] == data.drop_duplicates()

,k1,k2
0,True,True
2,True,True
3,True,True
6,True,True


#### Pode ser utilizado `drop_duplicates()` para eliminar duplicados em uma só coluna ou em um conjunto de colunas.

In [6]:
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,3
6,two,4


In [7]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,3,5
6,two,4,6


In [8]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


In [9]:
data.drop_duplicates(['k1', 'k2'])

,k1,k2,v1
0,one,1,0
2,one,2,2
3,two,3,3
6,two,4,6


### Mapear e transformar os dados.

#### A partir de um dicionário, é possível criar uma nova coluna para um Dataframe, onde as chaves são vinculadas a uma das séries e os valores fazem parte da nova coluna.

In [10]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 
                              'corned beef', 'Bacon', 'pastrami', 'honey ham', 
                              'nova lox'], 
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]
                    }
                   )
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


#### Podemos usar o método [`.unique()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.unique.html?highlight=unique#pandas.unique) para retornar dados por ordem de aparição.

In [11]:
#data.food.unique()
data['food'].unique()

array(['bacon', 'pulled pork', 'Pastrami', 'corned beef', 'Bacon',
       'pastrami', 'honey ham', 'nova lox'], dtype=object)

#### A ideia agora é atribuir um `animal` a cada tipo de `carne`. 

In [12]:
meat_to_animal = { 
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'}

#### Uma opção é fazer isso com os métodos [`.map()`](https://docs.python.org/3/library/functions.html#map), ela retorna um iterador que aplica uma função a cada item do iteravel, provendo o resultado.

In [13]:
pd.DataFrame(meat_to_animal.values(),index=meat_to_animal.keys()).reset_index().rename({0:'animal'},axis=1)

,index,animal
0,bacon,pig
1,pulled pork,pig
2,pastrami,cow
3,corned beef,cow
4,honey ham,pig
5,nova lox,salmon


In [14]:
data['animal'] = data['food'].apply(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### Substituir valores

#### O método [`.replace()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html) oferece várias maneiras de fazer substituições em uma série de Pandas: 

    1- Um valor antigo por um novo valor;
    2- Uma lista de valores antigos por um novo valor; 
    3- Uma lista de valores antigos por uma lista de valores novos; 
    4- Um dicionário que mapeia valores novos e antigos.

In [15]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [16]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [17]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

#### Usando um dicionário `dict`. 

In [18]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Discretizar e binarizar variáveis

#### O processo de transformar uma variável numérica em categórica se chama discretização. O método [`.cut()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html?highlight=cut#pandas.cut) retorna o intervalo semifechado ao qual cada entrada pertence.

In [19]:
ages = [26, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
data_2 = pd.DataFrame(ages, columns=['ages'])
data_2

,ages
0,26
1,22
2,25
3,27
4,21
5,23
6,37
7,31
8,61
9,45


In [20]:
# Definir os valores de corte
bins = [18, 25, 35, 60, 100]

# Obter uma lista de intervalos
data_2['cats'] = pd.cut(data_2['ages'], bins)
data_2

,ages,cats
0,26,"(25, 35]"
1,22,"(18, 25]"
2,25,"(18, 25]"
3,27,"(25, 35]"
4,21,"(18, 25]"
5,23,"(18, 25]"
6,37,"(35, 60]"
7,31,"(25, 35]"
8,61,"(60, 100]"
9,45,"(35, 60]"


In [21]:
data_2['cats'].value_counts()

(25, 35]     4
(18, 25]     4
(35, 60]     3
(60, 100]    1
Name: cats, dtype: int64

#### Podemos modificar a inclusão do valor de corte nos intervalos.

In [22]:
pd.cut(ages, [18, 26, 36, 61, 100], right = False)

[[26, 36), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

#### Podemos atribuir tags às categorias.

In [23]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
data_2['cats'] = pd.cut(data_2['ages'], bins, labels=group_names)

data_2

,ages,cats
0,26,YoungAdult
1,22,Youth
2,25,Youth
3,27,YoungAdult
4,21,Youth
5,23,Youth
6,37,MiddleAged
7,31,YoungAdult
8,61,Senior
9,45,MiddleAged


### Quantis em vez de intervalos preestabelecidos.

#### Vamos dividir em quantis, nesse caso, 10.

In [24]:
data = np.random.randn(1000)
cats = pd.qcut(data, 5) 
cats

[(-0.821, -0.249], (-0.821, -0.249], (-3.493, -0.821], (-0.821, -0.249], (-0.249, 0.238], ..., (-0.249, 0.238], (0.238, 0.836], (0.238, 0.836], (-3.493, -0.821], (0.238, 0.836]]
Length: 1000
Categories (5, interval[float64]): [(-3.493, -0.821] < (-0.821, -0.249] < (-0.249, 0.238] < (0.238, 0.836] < (0.836, 3.171]]

In [25]:
pd.value_counts(cats).sort_index()

(-3.493, -0.821]    200
(-0.821, -0.249]    200
(-0.249, 0.238]     200
(0.238, 0.836]      200
(0.836, 3.171]      200
dtype: int64

### Detectar e filtrar outliers

* A definição padrão de "`outlier`" determina que outliers são todos os valores que estão a mais de 3 desvios padrão acima ou abaixo da média.
* Obs: sempre que utilizar alguma função baseada em aleatoriedade (como no exemplo abaixo), garanta que a variável 'seed' está definida para algum valor fixo. Isso significa que o código irá gerar sempre o mesmo conjunto de valores aleatórios, permitindo que sua pesquisa apresente os mesmos resultados em outros momentos ou computadores.

In [26]:
np.random.seed(123)
data_3 = pd.DataFrame(np.random.randn(1000, 4))
data_3

,0,1,2,3
0,-1.085631,0.997345,0.282978,-1.506295
1,-0.578600,1.651437,-2.426679,-0.428913
2,1.265936,-0.866740,-0.678886,-0.094709
3,1.491390,-0.638902,-0.443982,-0.434351
4,2.205930,2.186786,1.004054,0.386186
...,...,...,...,...
995,-0.499897,0.587647,-0.926542,1.736982
996,-0.459550,0.125822,-1.119947,-0.521887
997,-2.013430,-0.028708,-0.103142,-1.761313
998,-0.185167,0.504077,1.354567,-0.907952


In [27]:
data_3.sample(5)

,0,1,2,3
468,0.181974,0.572843,-0.839113,0.192449
373,0.997890,1.686037,-0.794252,0.215802
280,1.655773,0.481287,-0.310228,-0.552144
764,-0.721146,0.389152,0.805637,1.407747
369,1.836869,-0.718403,1.134945,1.549722


In [28]:
np.random.seed(123)
data_3.sample(5)

,0,1,2,3
131,-0.448392,0.412819,0.600883,-1.131641
203,-1.415519,1.629611,1.052401,-0.148405
50,0.703310,-0.598105,2.200702,0.688297
585,2.156086,0.275040,-0.174344,-0.714881
138,-0.121741,-1.762898,1.158069,-0.682765


In [29]:
summary = data_3.describe()
summary

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.007502,0.039160,-0.010286,0.024285
std,0.977024,0.973484,1.012230,0.970421
min,-3.167055,-2.920029,-3.801378,-3.231055
25%,-0.662012,-0.636160,-0.687717,-0.599195
50%,-0.024843,0.062549,0.007035,0.038718
75%,0.613950,0.672448,0.664586,0.683228
max,3.050755,2.850708,2.766603,3.571579


In [30]:
mean = summary.loc['mean',3]
std = summary.loc['std',3]
threshold_min = mean - 3*std
threshold_max = mean + 3*std

In [31]:
print(round(mean,2), round(std,2), round(threshold_min,2), round(threshold_max,2))

0.02 0.97 -2.89 2.94


In [32]:
mask = (data_3[3] > threshold_min) & (data_3[3] < threshold_max)
data_3.loc[mask,: ]

,0,1,2,3
0,-1.085631,0.997345,0.282978,-1.506295
1,-0.578600,1.651437,-2.426679,-0.428913
2,1.265936,-0.866740,-0.678886,-0.094709
3,1.491390,-0.638902,-0.443982,-0.434351
4,2.205930,2.186786,1.004054,0.386186
...,...,...,...,...
995,-0.499897,0.587647,-0.926542,1.736982
996,-0.459550,0.125822,-1.119947,-0.521887
997,-2.013430,-0.028708,-0.103142,-1.761313
998,-0.185167,0.504077,1.354567,-0.907952


## Exercício: mostre as linhas que possuem outliers na coluna 3

In [33]:
mask_inverse = ~((data_3[3] > threshold_min) & (data_3[3] < threshold_max))
data_3.loc[mask_inverse,: ]

,0,1,2,3
48,0.199582,-0.126118,0.197019,-3.231055
64,0.590704,0.115299,0.029643,2.958625
182,0.272735,0.425336,-0.230904,3.571579


## Exercício: mostre as linhas que possuem outliers em pelo menos uma das colunas

In [42]:
data_3.loc[\
           data_3.apply(\
                        lambda col: (col[np.abs(col) > np.mean(col) + 3 * np.std(col)])\
                        .notnull()).fillna(False).index
           ,:]

,0,1,2,3
48,0.199582,-0.126118,0.197019,-3.231055
64,0.590704,0.115299,0.029643,2.958625
182,0.272735,0.425336,-0.230904,3.571579
235,-3.167055,-0.713989,-1.112364,-1.254184
395,-2.930223,1.035263,0.562861,0.812812
409,0.151037,0.069403,-3.801378,-1.127172
423,0.231228,1.076113,-3.587494,1.148869
510,0.506533,0.644099,-3.066988,-1.349275
910,3.050755,0.296552,-0.481843,0.930787


### PARTE II: Variáveis categóricas e Dummies


#### O uso de variáveis dummies, também conhecido como "one hot encoding", pode ser interpretado como o processo inverso da discretização. Nesse caso, pegamos variáveis categóricas e as transformamos em variáveis numéricas que seguem uma distribuição binomial com probabilidade p, onde p é a quantidade de vezes que a categoria aparece sobre o total de dados.

In [ ]:
df = pd.DataFrame({'cat_produto': ['b', 'b', 'a', 'c', 'a', 'b'], 
                   'cod_venda': np.arange(100, 112, 2)}
                 )
df

#### Pandas conta com o método [`.get_dummies()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) que recebe uma Série ou uma lista de Séries e realiza o one hot encoding.

#### Lembre que uma variável com `k` categorias pode ser representada com `k-1` variáveis. Por isso que um parâmetro-chave de pd.get_dummies é drop_first = True que gera k-1 categorias em vez de k.

In [ ]:
pd.get_dummies(df['cat_produto'])

#### Adicionamos um prefixo para identificar a categoria

In [ ]:
dummies = pd.get_dummies(df['cat_produto'], 
                         prefix = 'cat_produto',
#                         drop_first = True
                        )

dummies

#### Concatenamos a coluna `cod_venda`.

In [ ]:
df_with_dummy = df.join(dummies)
df_with_dummy

## Manipulação de strings

### String object methods

#### O método [`.split()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html?highlight=split#pandas.Series.str.split) pega uma string, divide-a de acordo com um delimitador (`sep`) e retorna uma lista

In [2]:
val = 'a,b, guido, asjd, kle'
val.split(',')

['a', 'b', ' guido', ' asjd', ' kle']

#### `strip()` pega uma string e retorna uma string sem os espaços iniciais e finais.

In [3]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido', 'asjd', 'kle']

In [4]:
val

'a,b, guido, asjd, kle'

In [5]:
'guido' in val

True

#### O método [`.find()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.find.html?highlight=find#pandas.Series.str.find) retorna o menor índice dentro de uma string na qual uma substring se encontra. Se não a encontra, retorna -1

In [6]:
val.find(':')

-1

#### O método [`.index ()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.html?highlight=index#pandas.Index) é semelhante, mas retorna um `ValueError` quando a substring procurada não é encontrada

In [7]:
val.index(',')

1

#### Se não encontra a substring, gera um erro.

In [8]:
val.index(':')

ValueError: substring not found

#### O mesmo exemplo acima, mas usando exceções. Permite-nos lidar com erros no tempo de execução.

In [ ]:
try:
    val.index(':')
except ValueError:
    print("Erro, substring não encontrada!")

#### O método [`.count()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.count.html?highlight=count#pandas.DataFrame.count) conta a ocorrência de uma substring determinada em uma string maior.

In [ ]:
val.count(',')

#### O método [`.replace()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html?highlight=replace#pandas.DataFrame.replace) substitui uma substring por outra.

In [ ]:
val.replace(',', ';')

### Funções vetorizadas para strings em Pandas

In [ ]:
import re
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com'}

data = pd.Series(data)

In [ ]:
data

In [ ]:
pattern_1 = "steve"
pattern_2 = ["steve","dave"]
pattern_3 = "\w"
pattern_4 = "\w+"

#### Já o método [`.findall()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.findall.html?highlight=findall#pandas.Series.str.findall) encontra todas as ocorrências de padrão ou expressões regulares em um objeto Series/Index.


In [ ]:
data.str.findall(pattern_1)

In [ ]:
data.str.findall(pattern_2)

In [ ]:
data.str.findall(pattern_3)

In [ ]:
data.str.findall(pattern_4)

#### O método [`.str.match()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.match.html?highlight=str%20match#pandas.Series.str.match) determina se cada string coincide com uma expressão regular.

In [ ]:
data[data.str.match(pattern_1)]

In [ ]:
#matches.str[0]
data.str[:5]

### Exemplo: Dataset movies

In [ ]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('movies.csv', 
                       header = None, 
                       names = mnames, 
                       encoding = "latin9", 
                       sep = ';'
                      )
movies[:10]

#### Subdividimos os gêneros.

In [ ]:
movie_genres_split = movies.genres.str.split('|').values
movie_genres_split

#### Criando uma lista de gêneros.

In [ ]:
genres = set([item for s in movie_genres_split for item in s])
genres

#### Criamos e codificamos as categorias como dummies. Escreve um 1 onde for correspondente.

In [ ]:
dummies = pd.DataFrame(np.zeros((len(movies), 
                                 len(genres))
                                , dtype = int
                               ), 
                       columns = genres
                      )
dummies.head()

In [ ]:
for i, gen in enumerate(movies.genres):
    if i<10:
        print(i, gen)
    dummies.loc[i, gen.split('|')] = 1

In [ ]:
dummies.head()

In [ ]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.head()

## Exercício: mostre os filmes do gênero aventura